In [2]:
# !/usr/bin/env python
from matplotlib.ticker import NullFormatter  # useful for `logit` scale
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import PySimpleGUI as sg
import matplotlib
matplotlib.use('TkAgg')

import PySimpleGUI as sg
import os.path


import PyPDF2


import nltk
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords


from collections import OrderedDict
import pprint

import pandas as pd
import numpy as np

import os


import tensorflow as tf
import tensorflow_hub as hub

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

from scrapy import cmdline
import json
from pandas.io.json import json_normalize


#--------------------------------pdf downloader import ----------------------------
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager #updated
import requests
import csv
from datetime import datetime, timedelta
import time
import calendar

import urllib.request
import shutil

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re

from IPython.display import display_html

def env_import():
    from matplotlib.ticker import NullFormatter  # useful for `logit` scale
    import matplotlib.pyplot as plt
    import numpy as np
    from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
    import PySimpleGUI as sg
    import matplotlib
    matplotlib.use('TkAgg')

    import PySimpleGUI as sg
    import os.path


    import PyPDF2


    import nltk
    from nltk.corpus import PlaintextCorpusReader
    from nltk.tokenize import word_tokenize
    from nltk.tokenize import sent_tokenize
    from nltk.probability import FreqDist
    from nltk.corpus import stopwords


    from collections import OrderedDict
    import pprint

    import pandas as pd
    import numpy as np

    import os


    import tensorflow as tf
    import tensorflow_hub as hub

    from wordcloud import WordCloud, STOPWORDS
    import matplotlib.pyplot as plt

    from scrapy import cmdline
    import json
    from pandas.io.json import json_normalize


#--------------------------------pdf downloader import ----------------------------
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager #updated
import requests
import csv
from datetime import datetime, timedelta
import time
import calendar

import urllib.request
import shutil

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re

from IPython.display import display_html




#---------------------------------End of import ------------------------------------

def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
#-------------PDF downloader function -----------------------
class pdfDownloader:
    def __init__(self, json_file = 'items.json'):
        self.file_path = json_file
        pass

    def download(self):
        pdf_count = 0
        data_str = open(self.file_path).read()
        data_str='['+data_str[:-2]+']'
        data_list = json.loads(data_str)

        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--headless")
        browser = webdriver.Chrome(ChromeDriverManager().install(), chrome_options = chrome_options)
        for item in data_list:
            company_name = item['company']
            comp_initial = item['url'].split('/')[2]
            index = comp_initial.find('.')
            comp_initial = comp_initial[index+1:]

            current_directory = os.getcwd()
            final_directory = os.path.join(current_directory, 'reports/'+company_name)
            if not os.path.exists(final_directory):
               os.makedirs(final_directory)

            browser.get(item['url'])
            links = get_links_onpage(browser)
            for lin in links:
                if ('pdf' in lin) and (comp_initial in lin):
                    pdf_count += 1
                    cur_file = os.listdir(final_directory)
                    if lin.split('/')[-1][:-4] + '.pdf' not in cur_file:
                        try:
                            download_file(lin, final_directory, lin.split('/')[-1][:-4])
                        except:
                            continue
                    else:
                        continue
        browser.quit()

        #searching for additional reports
        for item in data_list:
            company_name = item['company']
            break


        comp_initial = item['url'].split('/')[2]
        search_string = comp_initial.split('.')[1]
        comp_name = search_string

        #print(comp_name)
        # This is done to structure the string
        # into search url.(This can be ignored)
        search_string = search_string + " sustainability report"
        search_string = search_string.replace(' ', '+')

        # Assigning the browser variable with chromedriver of Chrome.
        # Any other browser and its respective webdriver
        # like geckodriver for Mozilla Firefox can be used
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        browser = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)

        for i in range(1):
            matched_elements = browser.get("https://www.google.com/search?q=" + search_string + "&start=" + str(i))

        links_fst = get_links_onpage(browser)

        google_website_visited = 0
        ##entering the google website
        while google_website_visited < 1:
            for lin in links_fst:
                pdf_exist = 0
                google_website_visited += 1
                download_count = 0
                if "google" in lin:
                    continue
                elif (comp_name + ".com" in lin and 'sustainability' in lin.split('/')[-2]) or\
                (comp_name + ".com" in lin and 'responsibilities' in lin.split('/')[-2]) or\
                ('our' in lin and 'sustainability' in lin.split('/')[-2]) or\
                (comp_name + ".com" in lin and 'sustainable' in lin.split('/')[-2]) or \
                (comp_name + ".com" in lin and 'sustainability' in lin.split('/')[-1]) or\
                (comp_name + ".com" in lin and 'csr' in lin.split('/')[-2]): #last one for at&t
                    browser.get(lin)
                    time.sleep(2)
                    ##entering the companies website
                    links_snd = get_links_onpage(browser)
                    for lin in links_snd:
                        if 'pdf' in lin:
                            #print(lin)
                            pdf_exist = 1
                            cur_file = os.listdir(final_directory)
                            if lin.split('/')[-1][:-4] + '.pdf' not in cur_file:
                                try:
                                    download_file(lin, final_directory, lin.split('/')[-1][:-4])
                                    download_count += 1
                                except:
                                    continue
                            else:
                                continue
                    if download_count > 0 or pdf_exist == 1:
                        browser.quit()
                        break
                    else:
                        for lin in links_snd:
                            if 'report' in lin:
                                #print(lin)
                                browser.get(lin)
                                links_trd = get_links_onpage(browser)
                                for lin in links_trd:
                                    if 'pdf' in lin:
                                        cur_file = os.listdir(final_directory)
                                        if lin.split('/')[-1][:-4] + '.pdf' not in cur_file:
                                            try:
                                                download_file(lin, final_directory, lin.split('/')[-1][:-4])
                                                download_count += 1
                                            except:
                                                continue
                                        else:
                                            continue
                                if download_count > 0 or pdf_exist == 1:
                                    browser.quit()
                                    break
                    break
            break

        if download_count == 0:
            browser.quit()
            #print('no new report found for the company :( !')

        pass


##utility functions

class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"

#opener = AppURLopener()
#response = opener.open('http://httpbin.org/user-agent')


def download_file(download_url, final_directory, filename):
    try:
        response = urllib.request.urlopen(download_url)
    except:

        opener = AppURLopener()
        response = opener.open(download_url)
    file = open(final_directory + '/' + filename + ".pdf", 'wb')
    file.write(response.read())
    file.close()

def get_links_onpage(driver):
    links = []
    elems = driver.find_elements_by_xpath("//a[@href]")
    for elem in elems:
        lin = elem.get_attribute("href")
        links.append(lin)
    return links



#-------------WordCloud functions -------------------------
DELETE_WORDS = []
def remove_words(text_string,DELETE_WORDS=DELETE_WORDS):
    for word in DELETE_WORDS:
        text_string = text_string.replace(word,' ')
    return text_string

#Remove short words
MIN_LENGTH = 0
def remove_short_words(text_string,min_length = MIN_LENGTH):
    word_list = text_string.split()
    for word in word_list:
        if len(word) < min_length:
            text_string = text_string.replace(' '+word+' ',' ',1)
    return text_string

def import_keyword(file):
    import csv
    with open(file,newline = '') as f:
        reader = csv.reader(f)
        data = list(reader)
        data.pop(0)

    return data

def wordcloud_drawer(fname, perfect_text ='no text'):

    if perfect_text == 'no text':
        f=open(fname,"r")
        temp = f.read().splitlines()
        text = ' '.join(temp)
    else:
        text = perfect_text

    text_string = remove_words(text)
    text_string = remove_short_words(text_string)
    wordcloud = WordCloud(stopwords=STOPWORDS,background_color='white',width=500,height=500).generate(text_string)

    wordcloud.to_file('wordcloud.png')

def keywordcloud_drawer(fname, perfect_text ='no text'):

    if perfect_text == 'no text':
        f=open(fname,"r")
        temp = f.read().splitlines()
        text = ' '.join(temp)
    else:
        text = perfect_text

    text_string = remove_words(text)
    text_string = remove_short_words(text_string)

    keyword = import_keyword('keyword.csv')
    words = nltk.Text(word_tokenize(text_string))
    result = list()
    for word in words:
        for kw in keyword:
            if str(word) == str(kw[0]):
                result.append(str(kw[0]))
    clean_up_string = ' '.join(word for word in result)
    wordcloud = WordCloud(stopwords=STOPWORDS,background_color='white',width=500,height=500).generate(clean_up_string)
    wordcloud.to_file('keywordcloud.png')

#--------------------------------PDF convertor function ------------------
def read_in_pdf(input_name):
    pdffileobj=open(input_name,'rb')
    pdfreader=PyPDF2.PdfFileReader(pdffileobj, strict=False)
    x=pdfreader.numPages
    try:
        os.remove("intermediate_file1.txt")
    except OSError:
        pass
    for i in range(x):
        pageobj=pdfreader.getPage(i)
        text=pageobj.extractText()

        file1=open(r"intermediate_file1.txt","a", encoding="utf-8")
        file1.writelines(text)
        file1.close()
    f=open(r"intermediate_file1.txt","r", encoding="utf-8")
    temp = f.read().splitlines()
    text = ' '.join(temp)
    return text

#---------------------------Sentence Embedding --------------------------



def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

# text=read_in_pdf('coty_sustainability_report_fy20.pdf')


def sentence_embedding(input_query,fname, perfect_text ='no text'):


    if perfect_text == 'no text':
        f=open(fname,"r")
        temp = f.read().splitlines()
        text = ' '.join(temp)
    else:
        text = perfect_text


    sentences = nltk.Text(sent_tokenize(text))
    words = nltk.Text(word_tokenize(text))

    tokenized_sent = []
    for s in sentences:
        tokenized_sent.append(word_tokenize(s.lower()))

    module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
    model = hub.load(module_url)
    # print ("module %s loaded" % module_url)

    sentence_embeddings = model(sentences)
    query = input_query
    query_vec = model([query])[0]

    df= pd.DataFrame()

    sentence_list = list()
    sim_list= list()

    for sent in sentences:
        sim = cosine(query_vec, model([sent])[0])
        sentence_list.append(sent)
        sim_list.append(sim)

    df['sentence'] = sentence_list
    df['similarity'] = sim_list
    pd.set_option('display.max_colwidth', 500)
    df =df.sort_values(by=['similarity'],ascending = False)

    return(df['sentence'].head(5))



# ------------------------------- Beginning of GUI CODE -------------------------------

# define the window layout
sg.change_look_and_feel('GreenMono')
file_list_column = [
    [
        sg.Text("Sustainability Folder"),
        sg.In(size=(25, 1), enable_events=True, key="-FOLDER-"),
        sg.FolderBrowse(),
    ],
    [
        sg.Listbox(
            values=[], enable_events=True, size=(40, 10), key="-FILE LIST-"
        )
    ],
    [   sg.Text("Input query"),
        sg.Input(key='_IN_')],
    [
        sg.Output(size=(60,15))
    ],

    [
        sg.Button("Run")
    ]
]

plot_column = [
          [sg.Text("Input Company"),
           sg.Input(key='_IN3_'),
           sg.Button('DownloadPDF'),
           sg.Button('Downloader')],
          [sg.Text('WordCloud')],
          [sg.Image(key='-IMAGE-')],
          [sg.Button('WordCloud'),sg.Button('Keyword Cloud')]
          ]


layout = [
    [
        sg.Column(file_list_column),
        sg.VSeperator(),
        sg.Column(plot_column),
    ]
]
# create the form and show it without the plot
window = sg.Window('Sustainability Report Interface Beta version', layout,size=(1270, 550), grab_anywhere=True, resizable=True,finalize=True, element_justification='center', font='Helvetica 14')

# add the plot to the window
# fig_canvas_agg = draw_figure(window['-CANVAS-'].TKCanvas, fig)


# ----------------------sustainability functions
# def read_in_pdf(input_name):
#     pdffileobj=open(input_name,'rb')
#     pdfreader=PyPDF2.PdfFileReader(pdffileobj, strict=False)
#     x=pdfreader.numPages
#     try:
#         os.remove("intermediate_file1.txt")
#     except OSError:
#         pass
#     for i in range(x):
#         pageobj=pdfreader.getPage(i)
#         text=pageobj.extractText()
#
#         file1=open(r"intermediate_file1.txt","a")
#         file1.writelines(text)
#         file1.close()
#     f=open(r"intermediate_file1.txt","r")
#     temp = f.read().splitlines()
#     text = ' '.join(temp)
#     return text


print("Welcome to our Sustainability Radar!")
print("New features will be added soon!")
print("Enjoy!")
print('--------------------------------------------------------------------------------')

# -------Conditional Statement Starts Here --------------------------------------------
count = 0
while True:

    event, values = window.read()
    if event == "Exit" or event == sg.WIN_CLOSED:
        break

# -------Directory Section ----------
    if event == "-FOLDER-":
        folder = values["-FOLDER-"]
        try:
            # Get list of files in folder
            file_list = os.listdir(folder)
        except:
            file_list = []

        fnames = [
            f
            for f in file_list
            if os.path.isfile(os.path.join(folder, f))
            and f.lower().endswith((".pdf", ".txt"))
        ]
        window["-FILE LIST-"].update(fnames)
    elif event == "-FILE LIST-":  # A file was chosen from the listbox
        try:
            filename = os.path.join(
                values["-FOLDER-"], values["-FILE LIST-"][0]
            )
            #window["-TOUT-"].update(filename)
            print("Current selected file: "+filename)
            selected_file = filename
        except:
            pass
#---------- sentence embedding ----------
    if event == "Run":
        # text=read_in_pdf('coty_sustainability_report_fy20.pdf')
        # sentences = nltk.Text(sent_tokenize(text))
        # print(len(sentences))
        try:
            print("####################################################")
            if 'pdf' in str(selected_file):
                from_pdf_to_txt = read_in_pdf(str(selected_file))
                SE = sentence_embedding(input_query = values['_IN_'],fname = str(selected_file),perfect_text = from_pdf_to_txt)
            else:
                SE = sentence_embedding(input_query = values['_IN_'],fname = str(selected_file))
            sent_count = 0
            for i in SE:
                print('Sentence '+str(sent_count)+': '+i)
                sent_count += 1
#         print(sentence_embedding(input_query = values['_IN_']))
            print("####################################################")
        except:
            print("Please choose the file to process.")
        # print(sentence_embedding())
        # if window:
        #     window.Refresh()

# -------WordCloud Section-------------
    if event == "WordCloud":
        try:
            if 'pdf' in str(selected_file):
                from_pdf_to_txt = read_in_pdf(str(selected_file))
                wordcloud_drawer(str(selected_file),perfect_text = from_pdf_to_txt)
            else:
                wordcloud_drawer(str(selected_file))
            window["-IMAGE-"].update(filename='wordcloud.png')
        except:
            print("Please choose the file to process.")
    if event == "Keyword Cloud":
        try:
            if 'pdf' in str(selected_file):
                from_pdf_to_txt = read_in_pdf(str(selected_file))
                keywordcloud_drawer(str(selected_file),perfect_text = from_pdf_to_txt)
            else:
                keywordcloud_drawer(str(selected_file))
            window["-IMAGE-"].update(filename='keywordcloud.png')
        except:
            print("Please choose the file to process.")


    if event == "DownloadPDF":
        try:
            print('###########################################')
            print('Downloading')
            print('This process may take more than 10 minutes.')
#             try:
#                 cmdline.execute('scrapy crawl Sustn'.split())
#             except:
#                 print("Json already downloaded")


            p = pdfDownloader()
            p.download()
            print('Downloading Finished')
        except:
            print('Download Error!')

    if event == "Downloader":
        os.system('python3 PDF_scrappy.py')


#     try:
#         os.rmdir('keywordcloud.png')
#         os.rmdir('wordcloud.png')
#     except OSError as e:
#         print("Error: %s : %s" % (dir_path, e.strerror))
window.close()
